In [70]:
from colmap_testing.colmap_helpers import read_write_model
import os
import cv2
import numpy as np

In [71]:
db_folder = "colmap_ws_aegis_room"
db_folder = "colmap_ws_meeting_room"

path_to_images = os.path.join("colmap_testing", db_folder,"images")
image_names = os.listdir(path_to_images)
image_names = sorted([int(os.path.splitext(name)[0]) for name in image_names])


In [72]:
path_to_model = os.path.join("colmap_testing", db_folder, "dense","0","sparse")
#path_to_images = os.path.join("colmap_testing", "colmap_ws_aegis_room", "dense", "0", "images")
#image_names = os.listdir(path_to_images)
#image_names = sorted([int(os.path.splitext(name)[0]) for name in image_names])
image_paths = [os.path.join(path_to_images, str(name) + ".png") for name in image_names]

rgb_images = np.array([np.array(cv2.imread(imagepath)) for imagepath in image_paths])
colmap_model = read_write_model.read_model(path_to_model, ".bin")

#cameras:fx, fy, cx, cy, width, height
#images:camera_id, qvec, tvec, name 
#points:xyz, rgb, error, image_ids, point2D_idxs
cameras, images, points = read_write_model.read_model(path_to_model, ".bin")


# 3D point list with one line of data per point:
# POINT3D_ID, X, Y, Z, R, G, B, ERROR, TRACK[] as (IMAGE_ID, POINT2D_IDX)
points_keys = list(points.keys())
z = 0
while z < 20:
    random_key = np.random.choice(points_keys)
    random_point = points[random_key]
    random_point_3d = random_point.xyz
    z = random_point_3d[2]
    print(random_point)

# Image list with two lines of data per image:
#   IMAGE_ID, QW, QX, QY, QZ, TX, TY, TZ, CAMERA_ID, NAME
#   POINTS2D[] as (X, Y, POINT3D_ID)

first_image = images[1]

# Camera list with one line of data per camera:
#   CAMERA_ID, MODEL, WIDTH, HEIGHT, PARAMS[]
first_camera = cameras[1]



#The coordinates of the projection/camera center are given by -R^t * T, 
# where R^t is the inverse/transpose of the 3x3 rotation matrix composed from the quaternion and T is the translation vector. 
# The local camera coordinate system of an image is defined in a way that the X axis points to the right, 
# the Y axis to the bottom, and the Z axis to the front as seen from the image.

#calculate the projection/camera center

Point3D(id=4811, xyz=array([ -8.78716416,  -1.03744484, 261.80014018]), rgb=array([72, 71, 66]), error=array(0.68049642), image_ids=array([10, 11,  6]), point2D_idxs=array([664, 723, 586]))


In [73]:

#sort keys by name in the dictionary images
def sort_keys_by_name(images):
    keys = images.keys()

    #get each name from the images
    names = []
    for key in keys:
        names.append(images[key].name)
    #remove the extension from the name
    names = [int(os.path.splitext(name)[0]) for name in names]
    sorted_idx = np.argsort(names)
    keys = np.array(list(keys))[sorted_idx]
    return keys

In [74]:
def get_frames(images, cameras, scale=1):
    frames = []
    keys = sort_keys_by_name(images)
    
    for key in keys:
        img = images[key]
        print(img.name)
        # rotation
        R = read_write_model.qvec2rotmat(img.qvec)

        # translation
        t = img.tvec

        # invert
        t = -R.T @ t
        R = R.T

        # intrinsics
        cam = cameras[img.camera_id]
        if cam.model in ("SIMPLE_PINHOLE", "SIMPLE_RADIAL", "RADIAL"):
            fx = fy = cam.params[0]
            cx = cam.params[1]
            cy = cam.params[2]
        elif cam.model in ("PINHOLE", "OPENCV", "OPENCV_FISHEYE", "FULL_OPENCV"):
            fx = cam.params[0]
            fy = cam.params[1]
            cx = cam.params[2]
            cy = cam.params[3]
        else:
            raise Exception("Camera model not supported")

        # intrinsics
        K = np.identity(3)
        K[0, 0] = fx
        K[1, 1] = fy
        K[0, 2] = cx
        K[1, 2] = cy
        frames.append((R, t, K))
    return frames

#frames are composed of R, t, K
frames = get_frames(images, cameras)

3.png
72.png
145.png
218.png
291.png
364.png
437.png
510.png
583.png
656.png
729.png


In [75]:
R, t, K = frames[0]
#add nested to t
t = np.array([t]).T
print(R)
print(t)
print(np.hstack((R, t)))

[[ 0.95587225 -0.15886082  0.24712644]
 [ 0.17090655  0.98489107 -0.02793806]
 [-0.23895436  0.06894075  0.9685804 ]]
[[-0.06315274]
 [-0.68384528]
 [ 3.27146292]]
[[ 0.95587225 -0.15886082  0.24712644 -0.06315274]
 [ 0.17090655  0.98489107 -0.02793806 -0.68384528]
 [-0.23895436  0.06894075  0.9685804   3.27146292]]


In [77]:
#project 3d point using the camera projection matrix onto the rgb image
#project them all into the same image
from matplotlib import pyplot as plt

if not os.path.exists(os.path.join("colmap_testing","reprojections",db_folder)):
    os.makedirs(os.path.join("colmap_testing","reprojections",db_folder))
subplots = []
for i,frame in enumerate(frames):
    R, t, K = frame
    #add nested to t
    t = np.array([t]).T
    P = K @ np.hstack((R, t)) #projection matrix
    point_on_image = P @ np.hstack((random_point_3d, 1))
    point_on_image = point_on_image / point_on_image[2]
    print(point_on_image)
    plt.scatter(1080-point_on_image[0], 1080-point_on_image[1], c="r")
    plt.imshow(rgb_images[i])
    plt.savefig(os.path.join("colmap_testing","reprojections",db_folder,"{}.png".format(i)))
    #plt.show()
    plt.close()
    

    



[706.83484621 485.89160513   1.        ]
[304.31226295 399.89888862   1.        ]
[922.47975119 653.90406011   1.        ]
[703.45470264 448.68744292   1.        ]
[230.41377389 479.77508511   1.        ]
[451.30286683 527.87474906   1.        ]
[700.83182669 500.55322808   1.        ]
[103.00425722 467.79121746   1.        ]
[735.67052195 526.06941147   1.        ]
[410.76470838 471.25045467   1.        ]
[451.27231417 535.60344041   1.        ]
